<a href="https://colab.research.google.com/github/PODEE/PYOMO-Optimization-Problems/blob/main/Bland_Brewery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import logging, pyomo
import numpy as np

from pyomo.environ import *
from pyomo.opt import SolverFactory, SolverStatus
from pyomo.util.infeasible import log_infeasible_constraints

In [ ]:
pyomo.__version__

'6.0.1'

### Bland Brewery optimization problem from Columbia Business School

http://www.columbia.edu/itc/sipa/U6033/client_edit/lectures/lec1.pdf

### CONCRETE MODEL

In [ ]:
model = ConcreteModel()

In [ ]:
# Define decision Variables
model.ALE = Var(within=NonNegativeIntegers)
model.BEER = Var(within=NonNegativeIntegers)

In [ ]:
# Define Objective Function
model.SystemProfit = Objective(expr = 13*model.ALE + 23*model.BEER, sense = maximize)

In [ ]:
# Define Constraints
model.corn_lbs = Constraint(expr = 5*model.ALE + 15*model.BEER <= 480)
model.hops_lbs = Constraint(expr = 4*model.ALE + 4*model.BEER <= 160)
model.barley_lbs = Constraint(expr = 35*model.ALE + 20*model.BEER <= 1190)

In [ ]:
# define glpk path
solvername = 'glpk'
solverpath_folder = '/data/conda/envs/pk30901/davon/bin/'
solverpath_exe = '/data/conda/envs/pk30901/davon/bin/glpsol' 

opt = SolverFactory(solvername,executable=solverpath_exe)

In [ ]:
# print system info
result_obj = opt.solve(model, tee=True)
model.pprint()

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /projects/pk30901/tmp/tmp7jak1mmf.glpk.raw --wglp /projects/pk30901/tmp/tmp2telr6hw.glpk.glp
 --cpxlp /projects/pk30901/tmp/tmpc7wa60ny.pyomo.lp
Reading problem data from '/projects/pk30901/tmp/tmpc7wa60ny.pyomo.lp'...
4 rows, 3 columns, 7 non-zeros
2 integer variables, none of which are binary
34 lines were read
Writing problem data to '/projects/pk30901/tmp/tmp2telr6hw.glpk.glp'...
26 lines were written
GLPK Integer Optimizer, v4.65
4 rows, 3 columns, 7 non-zeros
2 integer variables, none of which are binary
Preprocessing...
3 rows, 2 columns, 6 non-zeros
2 integer variables, none of which are binary
Scaling...
 A: min|aij| =  4.000e+00  max|aij| =  3.500e+01  ratio =  8.750e+00
GM: min|aij| =  6.606e-01  max|aij| =  1.514e+00  ratio =  2.291e+00
EQ: min|aij| =  4.364e-01  max|aij| =  1.000e+00  ratio =  2.291e+00
2N: min|aij| =  3.125e-01  max|aij| =  1.094e+00  ratio =  3.500e+00
Constructing ini

In [ ]:
#status
print("Solve Status - ",result_obj.solver.termination_condition)
print("Objective Function Value = ", value(model.SystemProfit))

Solve Status -  optimal
Objective Function Value =  800.0


In [ ]:
# print out values of results
for v in model.component_objects(Var):
    print(v)
    for index in v:
        print(index, v[index].value)

ALE
None 12.0
BEER
None 28.0


### Abstract Model Formulation

In [ ]:
model = AbstractModel()

In [ ]:
produce_items = ['ALE', 'BEER']
ingredients = ['Corn', 'Hops', 'Barley_Malt']

Profit_dict = {'ALE':13, 'BEER':23}
max_pounds_dict = {'Corn':480, 'Hops':160, 'Barley_Malt':1190}

CORN = [5,15]
Hops = [4,4]
Hops = [35,20]

In [ ]:
# create a dictionary for amount of each ingredient needed for each PRODUCT
v={}
v['Corn','ALE'] = 5
v['Corn','BEER'] = 15
v['Hops','ALE'] = 4
v['Hops','BEER'] = 4
v['Barley_Malt','ALE'] = 35
v['Barley_Malt','BEER'] = 20

In [ ]:
#define sets

model.PRODUCT = Set(initialize = produce_items)
model.INGREDIENTS = Set(initialize = ingredients)

In [ ]:
#define parameters

model.max_pounds = Param(model.INGREDIENTS, within=NonNegativeIntegers, initialize = max_pounds_dict, default=0)
model.Profit = Param(model.PRODUCT, within=Any, initialize = Profit_dict, default=0)
model.Barrel_max = Param(model.INGREDIENTS, model.PRODUCT, initialize=v, default=0)

In [ ]:
# define decision variable
model.x = Var(model.PRODUCT, within=NonNegativeIntegers)

In [ ]:
#define objective

def objective(model):
    return sum(model.Profit[i]*model.x[i] for i in model.PRODUCT)

model.SystemProfit = Objective(rule=objective, sense=maximize)

In [ ]:
# for each ingredient, we want a constraint for the max lbs available
def max_lb_rule(model, i):
    return sum(model.x[j]*model.Barrel_max[i,j] for j in model.PRODUCT) <= model.max_pounds[i]

model.max_lbs = Constraint(model.INGREDIENTS, rule=max_lb_rule)

In [ ]:
#create instance of the abstract model
instance = model.create_instance()

In [ ]:
instance.PRODUCT.ordered_data()

('ALE', 'BEER')

In [ ]:
instance.INGREDIENTS.ordered_data()

('Corn', 'Hops', 'Barley_Malt')

In [ ]:
instance.max_pounds.items()

[('Corn', 480), ('Hops', 160), ('Barley_Malt', 1190)]

In [ ]:
instance.Profit.items()

[('ALE', 13), ('BEER', 23)]

In [ ]:
instance.Barrel_max.items()

[(('Corn', 'ALE'), 5),
 (('Corn', 'BEER'), 15),
 (('Hops', 'ALE'), 4),
 (('Hops', 'BEER'), 4),
 (('Barley_Malt', 'ALE'), 35),
 (('Barley_Malt', 'BEER'), 20)]

In [ ]:
#solve
result = opt.solve(instance, report_timing = True)

        0.00 seconds required to write file
        0.01 seconds required for presolve
        0.02 seconds required for solver
        0.00 seconds required to read logfile 
        0.00 seconds required to read solution file 
        0.02 seconds required for postsolve


In [ ]:
#status
print("Solve Status - ", result.solver.termination_condition)
print("Objective Function Value = ", value(instance.SystemProfit))

Solve Status -  optimal
Objective Function Value =  800.0


In [ ]:
#solver output
result.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 800.0
  Upper bound: 800.0
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 3
  Number of nonzeros: 7
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.022266626358032227
# ----------------------------------------------------------
#   Solution Information
# ------------------------------------

In [ ]:
instance.pprint()

3 Set Declarations
    Barrel_max_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain              : Size : Members
        None :     2 : INGREDIENTS*PRODUCT :    6 : {('Corn', 'ALE'), ('Corn', 'BEER'), ('Hops', 'ALE'), ('Hops', 'BEER'), ('Barley_Malt', 'ALE'), ('Barley_Malt', 'BEER')}
    INGREDIENTS : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'Corn', 'Hops', 'Barley_Malt'}
    PRODUCT : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'ALE', 'BEER'}

3 Param Declarations
    Barrel_max : Size=6, Index=Barrel_max_index, Domain=NonNegativeIntegers, Default=0, Mutable=False
        Key                     : Value
         ('Barley_Malt', 'ALE') :    35
        ('Barley_Malt', 'BEER') :    20
                ('Corn', 'ALE') :     5
               ('Corn', 'BEER') :    15
                ('Hops', 'ALE') :     4
 

In [ ]:
# print out values of results
for v in instance.component_objects(Var):
    print(v)
    for index in v:
        print(index, v[index].value)

x
ALE 12.0
BEER 28.0
